In [1]:
!pip install tensorflow==2.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.3/394.3 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.0 MB/s eta 0:00:00
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77048 sha256=469c4c9c3df0bac140e0c53e06c27257aa8d84d3702e30823b37788ad47cac28
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.14.1
    Uninstalling wrapt-1.14.1:
      Successfully uninstalled wrapt-1.14.1
  Attempting uninstall: typing

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend  as K
from glob import glob
import os

2022-07-24 12:21:04.498042: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
# Variables
IMGS_DIR = "../input/landcoverai/images/"
MASKS_DIR = "../input/landcoverai/masks/"
OUTPUT_DIR = "/kaggle/input/d/revanthvenkateswar/landcoverai/output/"

TARGET_SIZE = 256
NUM_CLASSES = 5
BATCH_SIZE = 15
epochs = 15
train_percent = 0.6
val_percent = 0.2
seed = 47

# 0 is background, 1 is building, 2 is woodland, 3 is water, 4 is road
class_weights = {0: 1/(1-(1.85+72.02+13.15+3.5)/216.27), 1: 1/(1.85/216.27), 2: 1/(72.02/216.27), 3: 1/(13.15/216.27), 4: 1/(3.5/216.27)}

In [4]:
# import cv2
# import os

# img_paths = glob(os.path.join(IMGS_DIR, "*.tif"))
# mask_paths = glob(os.path.join(MASKS_DIR, "*.tif"))

# img_paths.sort()
# mask_paths.sort()

# if not os.path.exists(OUTPUT_DIR + "images/"):
#     os.makedirs(OUTPUT_DIR + "images/")
#     os.makedirs(OUTPUT_DIR + "masks/")

# for i, (img_path, mask_path) in enumerate(zip(img_paths, mask_paths)):
#     img_filename = os.path.splitext(os.path.basename(img_path))[0]
#     mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
#     img = cv2.imread(img_path)
#     mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

#     assert img_filename == mask_filename and img.shape[:2] == mask.shape[:2]

#     k = 0
#     for y in range(0, img.shape[0], TARGET_SIZE):
#         for x in range(0, img.shape[1], TARGET_SIZE):
#             img_tile = img[y:y + TARGET_SIZE, x:x + TARGET_SIZE]
#             mask_tile = mask[y:y + TARGET_SIZE, x:x + TARGET_SIZE]

#             if img_tile.shape[0] == TARGET_SIZE and img_tile.shape[1] == TARGET_SIZE:
#                 out_img_name = "{}_{}.jpg".format(img_filename, k)
#                 out_mask_name = "{}_m_{}.png".format(mask_filename, k)
                
#                 out_img_path = os.path.join(OUTPUT_DIR+"images/", out_img_name)
#                 cv2.imwrite(out_img_path, img_tile)
#                 out_mask_path = os.path.join(OUTPUT_DIR+"masks/", out_mask_name)
#                 cv2.imwrite(out_mask_path, mask_tile)

#             k += 1

#     print("Processed {} {}/{}".format(img_filename, i + 1, len(img_paths)))

In [5]:
def load_data(path):
    image_paths = sorted(glob(os.path.join(path, "images/*")))
    mask_paths = sorted(glob(os.path.join(path, "masks/*")))
    return image_paths, mask_paths

def load_image_and_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image.set_shape([TARGET_SIZE, TARGET_SIZE, 3])

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    # # If we subtract 1 from mask,
    # # we ignore background in the one hot encoding 
    # # as tf.one_hot will convert -1 to [0, 0, 0, 0]
    # # and 0 as [1, 0, 0, 0]

    # mask = mask - 1 
    mask = tf.one_hot(mask, depth=NUM_CLASSES)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.squeeze(mask)
    mask.set_shape([TARGET_SIZE, TARGET_SIZE, NUM_CLASSES])
    return image, mask

def create_dataset(image_paths, mask_paths):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    dataset = dataset.shuffle(buffer_size=len(image_paths), seed=seed)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def test_train_val_split(full_dataset, size, train_percent=0.6, val_percent=0.2, batch_size=8):
    train_dataset = full_dataset.take(int(train_percent*size))
    train_dataset = train_dataset.batch(batch_size)
    test_dataset = full_dataset.skip(int(train_percent*size))

    val_dataset = test_dataset.take(int(val_percent*size))
    val_dataset = val_dataset.batch(batch_size)
    test_dataset = test_dataset.skip(int(val_percent*size))
    test_dataset = test_dataset.batch(batch_size)
    
    return train_dataset, val_dataset, test_dataset

In [6]:
image_paths, mask_paths = load_data(OUTPUT_DIR)
dataset = create_dataset(image_paths, mask_paths)
train, val, test = test_train_val_split(dataset, len(image_paths), train_percent, val_percent, BATCH_SIZE)

2022-07-24 12:21:08.648942: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-24 12:21:08.650143: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-24 12:21:08.717258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 12:21:08.717960: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-07-24 12:21:08.718027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-24 12:21:08.771747: I tensorflow/stream_executor/platform/def

In [7]:
def conv_bn(x, filters, kernel_size=3, strides=1):
    x = tf.keras.layers.Conv2D(filters=filters,
                               kernel_size=kernel_size,
                               strides=strides,
                               padding='same',
                               use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def dilated_conv_bn(x, filters, kernel_size=3, rate=1):
    x = tf.keras.layers.Conv2D(filters=filters,
                               kernel_size=kernel_size,
                               padding='same',
                               use_bias=True,
                               dilation_rate=(rate,rate))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def dilated_spatial_pyramidal_pooling_conv(x, filters, kernel_size=3):
    x2 = tf.keras.layers.Conv2D(filters=8,
                                kernel_size=3,
                                padding='same',
                                use_bias=True,
                                dilation_rate=(2,2))(x)
    x4 = tf.keras.layers.Conv2D(filters=8,
                                kernel_size=3,
                                padding='same',
                                use_bias=True,
                                dilation_rate=(4,4))(x)
    x8 = tf.keras.layers.Conv2D(filters=8,
                                kernel_size=6,
                                padding='same',
                                use_bias=True,
                                dilation_rate=(8,8))(x)
    x16 = tf.keras.layers.Conv2D(filters=8,
                                 kernel_size=9,
                                 padding='same',
                                 use_bias=True,
                                 dilation_rate=(8,8))(x)
    
    x = tf.keras.layers.Concatenate(axis=3)([x, x2, x4, x8, x16])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(filters=filters,
                               kernel_size=1,
                               padding='same',
                               use_bias=True)(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def model(x):
    # Encoder
    x = conv_bn(x, filters=16)
    x = conv_bn(x, filters=32)
    x1 = dilated_spatial_pyramidal_pooling_conv(x, filters=16)
    x = dilated_conv_bn(x, filters=64, rate=2)
    x = conv_bn(x, filters=32)
    x2 = dilated_spatial_pyramidal_pooling_conv(x, filters=16)
    x = dilated_conv_bn(x, filters=64, rate=4)
    x = conv_bn(x, filters=32)
    x4 = dilated_spatial_pyramidal_pooling_conv(x, filters=16)
    x = dilated_conv_bn(x, filters=64, rate=8)
    x = conv_bn(x, filters=32)
    x8 = dilated_spatial_pyramidal_pooling_conv(x, filters=16)
    x = dilated_conv_bn(x, filters=64, rate=16)
    x = conv_bn(x, filters=32)
    x = conv_bn(x, filters=32)

    # Decoder
    x = tf.keras.layers.Concatenate(axis=3)([x8, x])
    x = conv_bn(x, filters=32)
    x = tf.keras.layers.Concatenate(axis=3)([x4, x])
    x = conv_bn(x, filters=32)
    x = tf.keras.layers.Concatenate(axis=3)([x2, x])
    x = conv_bn(x, filters=32)
    x = tf.keras.layers.Concatenate(axis=3)([x1, x])
    x = conv_bn(x, filters=32)

    # Number of filters = number of classes
    x = tf.keras.layers.Conv2D(filters=NUM_CLASSES, kernel_size=1, strides=1, activation='softmax')(x)
    return x

In [8]:
# This file is taken 
# from https://github.com/mlyg/unified-focal-loss/blob/main/loss_functions.py
# Edited to make it compatible with multi-class segmentation

# Helper function to enable loss function to be flexibly used for 
# both 2D or 3D image segmentation - source: https://github.com/frankkramer-lab/MIScnn
def identify_axis(shape):
    # Three dimensional
    if len(shape) == 5 : return [1,2,3]
    # Two dimensional
    elif len(shape) == 4 : return [1,2]
    # Exception - Unknown
    else : raise ValueError('Metric: Shape of tensor is neither 2D or 3D.')

################################
#       Dice coefficient       #
################################
def dice_coefficient(delta = 0.5, smooth = 0.000001):
    """The Dice similarity coefficient, also known as the Sørensen–Dice index or simply Dice coefficient, is a statistical tool which measures the similarity between two sets of data.
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.5
    smooth : float, optional
        smoothing constant to prevent division by zero errors, by default 0.000001
    """
    def loss_function(y_true, y_pred):
        axis = identify_axis(y_true.get_shape())
        # Calculate true positives (tp), false negatives (fn) and false positives (fp)   
        tp = K.sum(y_true * y_pred, axis=axis)
        fn = K.sum(y_true * (1-y_pred), axis=axis)
        fp = K.sum((1-y_true) * y_pred, axis=axis)
        dice_class = (tp + smooth)/(tp + delta*fn + (1-delta)*fp + smooth)
        # Average class scores
        dice = K.mean(dice_class)

        return dice

    return loss_function

################################
#           Dice loss          #
################################
def dice_loss(delta = 0.5, smooth = 0.000001):
    """Dice loss originates from Sørensen–Dice coefficient, which is a statistic developed in 1940s to gauge the similarity between two samples.
    
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.5
    smooth : float, optional
        smoothing constant to prevent division by zero errors, by default 0.000001
    """
    def loss_function(y_true, y_pred):
        axis = identify_axis(y_true.get_shape())
        # Calculate true positives (tp), false negatives (fn) and false positives (fp)
        tp = K.sum(y_true * y_pred, axis=axis)
        fn = K.sum(y_true * (1-y_pred), axis=axis)
        fp = K.sum((1-y_true) * y_pred, axis=axis)
        # Calculate Dice score
        dice_class = (tp + smooth)/(tp + delta*fn + (1-delta)*fp + smooth)
        # Average class scores
        dice_loss = K.mean(1-dice_class)

        return dice_loss
        
    return loss_function


################################
#         Tversky loss         #
################################
def tversky_loss(delta = 0.7, smooth = 0.000001):
    """Tversky loss function for image segmentation using 3D fully convolutional deep networks
	Link: https://arxiv.org/abs/1706.05721
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.7
    smooth : float, optional
        smoothing constant to prevent division by zero errors, by default 0.000001
    """
    def loss_function(y_true, y_pred):
        axis = identify_axis(y_true.get_shape())
        # Calculate true positives (tp), false negatives (fn) and false positives (fp)   
        tp = K.sum(y_true * y_pred, axis=axis)
        fn = K.sum(y_true * (1-y_pred), axis=axis)
        fp = K.sum((1-y_true) * y_pred, axis=axis)
        tversky_class = (tp + smooth)/(tp + delta*fn + (1-delta)*fp + smooth)
        # Average class scores
        tversky_loss = K.mean(1-tversky_class)

        return tversky_loss

    return loss_function

################################
#          Combo loss          #
################################
def combo_loss(alpha=0.5,beta=0.5):
    """Combo Loss: Handling Input and Output Imbalance in Multi-Organ Segmentation
    Link: https://arxiv.org/abs/1805.02798
    Parameters
    ----------
    alpha : float, optional
        controls weighting of dice and cross-entropy loss., by default 0.5
    beta : float, optional
        beta > 0.5 penalises false negatives more than false positives., by default 0.5
    """
    def loss_function(y_true,y_pred):
        dice = dice_coefficient()(y_true, y_pred)
        # axis = identify_axis(y_true.get_shape())
        # Clip values to prevent division by zero error
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)

        if beta is not None:
            beta_weight = np.array([beta, 1-beta])
            cross_entropy = beta_weight * cross_entropy
        # sum over classes
        cross_entropy = K.mean(K.sum(cross_entropy, axis=[-1]))
        if alpha is not None:
            combo_loss = (alpha * cross_entropy) - ((1 - alpha) * dice)
        else:
            combo_loss = cross_entropy - dice
        return combo_loss

    return loss_function

################################
#      Focal Tversky loss      #
################################
def focal_tversky_loss(delta=0.7, gamma=0.75, smooth=0.000001):
    """A Novel Focal Tversky loss function with improved Attention U-Net for lesion segmentation
    Link: https://arxiv.org/abs/1810.07842
    Parameters
    ----------
    gamma : float, optional
        focal parameter controls degree of down-weighting of easy examples, by default 0.75
    """
    def loss_function(y_true, y_pred):
        # Clip values to prevent division by zero error
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon) 
        axis = identify_axis(y_true.get_shape())
        # Calculate true positives (tp), false negatives (fn) and false positives (fp)     
        tp = K.sum(y_true * y_pred, axis=axis)
        fn = K.sum(y_true * (1-y_pred), axis=axis)
        fp = K.sum((1-y_true) * y_pred, axis=axis)
        tversky_class = (tp + smooth)/(tp + delta*fn + (1-delta)*fp + smooth)
        # Average class scores
        focal_tversky_loss = K.mean(K.pow((1-tversky_class), gamma))
	
        return focal_tversky_loss

    return loss_function


################################
#          Focal loss          #
################################
def focal_loss(alpha=None, gamma_f=2.):
    """Focal loss is used to address the issue of the class imbalance problem. A modulation term applied to the Cross-Entropy loss function.
    Parameters
    ----------
    alpha : float, optional
        controls relative weight of false positives and false negatives. alpha > 0.5 penalises false negatives more than false positives, by default None
    gamma_f : float, optional
        focal parameter controls degree of down-weighting of easy examples, by default 2.
    """
    def loss_function(y_true, y_pred):
        # axis = identify_axis(y_true.get_shape())
        # Clip values to prevent division by zero error
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)

        if alpha is not None:
            alpha_weight = np.array(alpha, dtype=np.float32)
            focal_loss = alpha_weight * K.pow(1 - y_pred, gamma_f) * cross_entropy
        else:
            focal_loss = K.pow(1 - y_pred, gamma_f) * cross_entropy

        focal_loss = K.mean(K.sum(focal_loss, axis=[-1]))
        return focal_loss
        
    return loss_function

################################
#     Symmetric Focal loss     #
################################
def symmetric_focal_loss(delta=0.7, gamma=2.):
    """
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.7
    gamma : float, optional
        Focal Tversky loss' focal parameter controls degree of down-weighting of easy examples, by default 2.0
    """
    def loss_function(y_true, y_pred):

        # axis = identify_axis(y_true.get_shape())  

        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)

        #calculate losses separately for each class
        background_ce = K.pow(1 - y_pred[:,:,:,0], gamma) * cross_entropy[:,:,:,0]
        background_ce =  (1 - delta) * background_ce

        # This section is modified for multiclass segmentation
        numClasses = K.int_shape(y_pred)[-1]
        list_of_class_ce = [background_ce]
        
        for i in range(1, numClasses):
            class_ce = cross_entropy[:,:,:,i]
            class_ce = delta * class_ce
            list_of_class_ce.append(class_ce)

        loss = K.mean(K.sum(tf.stack(list_of_class_ce, axis=-1), axis=-1))

        return loss

    return loss_function

#################################
# Symmetric Focal Tversky loss  #
#################################
def symmetric_focal_tversky_loss(delta=0.7, gamma=0.75):
    """This is the implementation for multi-class segmentation.
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.7
    gamma : float, optional
        focal parameter controls degree of down-weighting of easy examples, by default 0.75
    """
    def loss_function(y_true, y_pred):
        # Clip values to prevent division by zero error
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        axis = identify_axis(y_true.get_shape())
        # Calculate true positives (tp), false negatives (fn) and false positives (fp)     
        tp = K.sum(y_true * y_pred, axis=axis)
        fn = K.sum(y_true * (1-y_pred), axis=axis)
        fp = K.sum((1-y_true) * y_pred, axis=axis)
        dice_class = (tp + epsilon)/(tp + delta*fn + (1-delta)*fp + epsilon)

        #calculate losses separately for each class, enhancing both classes
        background_dice = (1-dice_class[:,0]) * K.pow(1-dice_class[:,0], -gamma)

        # modify this section below for multiclass segmentation
        numClasses = K.int_shape(y_pred)[-1]
        list_of_class_dice = [background_dice]
        for i in range(1, numClasses):
            class_dice = (1-dice_class[:,i]) * K.pow(1-dice_class[:,i], -gamma)
            list_of_class_dice.append(class_dice)

        # Average class scores
        loss = K.mean(tf.stack(list_of_class_dice, axis=-1))
        return loss

    return loss_function


################################
#     Asymmetric Focal loss    #
################################
def asymmetric_focal_loss(delta=0.7, gamma=2.):
    """For Imbalanced datasets (multi-class)
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.7
    gamma : float, optional
        Focal Tversky loss' focal parameter controls degree of down-weighting of easy examples, by default 2.0
    """
    def loss_function(y_true, y_pred):
        axis = identify_axis(y_true.get_shape())  

        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)

        #calculate losses separately for each class, only suppressing background class
        background_ce = K.pow(1 - y_pred[:,:,:,0], gamma) * cross_entropy[:,:,:,0]
        background_ce =  (1 - delta) * background_ce

        # This section is modified for multiclass segmentation
        numClasses = K.int_shape(y_pred)[-1]
        list_of_class_ce = [background_ce]

        for i in range(1, numClasses):
            class_ce = cross_entropy[:,:,:,i]
            class_ce = delta * class_ce
            list_of_class_ce.append(class_ce)

        loss = K.mean(K.sum(tf.stack(list_of_class_ce, axis=-1), axis=-1))

        return loss

    return loss_function

#################################
# Asymmetric Focal Tversky loss #
#################################
def asymmetric_focal_tversky_loss(delta=0.7, gamma=0.75):
    """This is the implementation for multi-class segmentation.
    Parameters
    ----------
    delta : float, optional
        controls weight given to false positive and false negatives, by default 0.7
    gamma : float, optional
        focal parameter controls degree of down-weighting of easy examples, by default 0.75
    """
    def loss_function(y_true, y_pred):
        # Clip values to prevent division by zero error
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        axis = identify_axis(y_true.get_shape())
        # Calculate true positives (tp), false negatives (fn) and false positives (fp)     
        tp = K.sum(y_true * y_pred, axis=axis)
        fn = K.sum(y_true * (1-y_pred), axis=axis)
        fp = K.sum((1-y_true) * y_pred, axis=axis)
        dice_class = (tp + epsilon)/(tp + delta*fn + (1-delta)*fp + epsilon)

        #calculate losses separately for each class, enhancing both classes
        background_dice = (1-dice_class[:,0])

        # modify this section below for multiclass segmentation
        numClasses = K.int_shape(y_pred)[-1]
        list_of_class_dice = [background_dice]
        for i in range(1, numClasses):
            class_dice = (1-dice_class[:,i]) * K.pow(1-dice_class[:,i], -gamma)
            list_of_class_dice.append(class_dice)

        # Average class scores
        loss = K.mean(tf.stack(list_of_class_dice, axis=-1))

        return loss

    return loss_function


###########################################
#      Symmetric Unified Focal loss       #
###########################################
def sym_unified_focal_loss(weight=0.5, delta=0.6, gamma=0.5):
    """The Unified Focal loss is a new compound loss function that unifies Dice-based and cross entropy-based loss functions into a single framework.
    Parameters
    ----------
    weight : float, optional
        represents lambda parameter and controls weight given to symmetric Focal Tversky loss and symmetric Focal loss, by default 0.5
    delta : float, optional
        controls weight given to each class, by default 0.6
    gamma : float, optional
        focal parameter controls the degree of background suppression and foreground enhancement, by default 0.5
    """
    def loss_function(y_true,y_pred):
        symmetric_ftl = symmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        symmetric_fl = symmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        if weight is not None:
            return (weight * symmetric_ftl) + ((1-weight) * symmetric_fl)  
        else:
            return symmetric_ftl + symmetric_fl

    return loss_function

###########################################
#      Asymmetric Unified Focal loss      #
###########################################
def asym_unified_focal_loss(weight=0.5, delta=0.6, gamma=0.5):
    """The Unified Focal loss is a new compound loss function that unifies Dice-based and cross entropy-based loss functions into a single framework.
    Parameters
    ----------
    weight : float, optional
        represents lambda parameter and controls weight given to asymmetric Focal Tversky loss and asymmetric Focal loss, by default 0.5
    delta : float, optional
        controls weight given to each class, by default 0.6
    gamma : float, optional
        focal parameter controls the degree of background suppression and foreground enhancement, by default 0.5
    """
    def loss_function(y_true,y_pred):
        asymmetric_ftl = asymmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        asymmetric_fl = asymmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
        if weight is not None:
            return (weight * asymmetric_ftl) + ((1-weight) * asymmetric_fl)  
        else:
            return asymmetric_ftl + asymmetric_fl

    return loss_function

In [9]:
################################
#         OneHot MeanIoU       #
################################
class OneHotMeanIoU(tf.keras.metrics.MeanIoU):
    '''
    Custom metric to calculate OneHotMeanIoU
    as the keras version is not available in tf 2.6
    '''
    def __init__(
        self,
        num_classes: int,
        name=None,
        dtype=None,
    ):
        super(OneHotMeanIoU, self).__init__(
            num_classes=num_classes,
            name=name,
            dtype=dtype,
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        """Accumulates the confusion matrix statistics.
        Args:
          y_true: The ground truth values.
          y_pred: The predicted values.
          sample_weight: Optional weighting of each example. Defaults to 1. Can be a
            `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
            be broadcastable to `y_true`.
        Returns:
          Update op.
        """
        # Select max hot-encoding channels to convert into all-class format
        y_true = tf.argmax(y_true, axis=-1, output_type=tf.int32)
        y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)
        
        return super().update_state(y_true, y_pred, sample_weight)

In [10]:
# Initialize the model
input = tf.keras.layers.Input(shape=[TARGET_SIZE, TARGET_SIZE, 3])
output = model(input)
model = tf.keras.Model(input, output)

In [11]:
from tensorflow.keras.metrics import Precision, Recall
# 0 is background, 1 is building, 2 is woodland, 3 is water, 4 is road
model.compile(optimizer='adam', loss=asym_unified_focal_loss(), 
              metrics=[OneHotMeanIoU(NUM_CLASSES, name='MeanIoU'),
                       Precision(name='bgrPcsn', class_id=0),
                       Precision(name='bldPcsn', class_id=1),
                       Precision(name='wldPcsn', class_id=2),
                       Precision(name='wtrPcsn', class_id=3),
                       Precision(name='roadPcsn', class_id=4),
                       Recall(name='bgrRcl', class_id=0),
                       Recall(name='bldRcl', class_id=1),
                       Recall(name='wldRcl', class_id=2),
                       Recall(name='wtrRcl', class_id=3),
                       Recall(name='roadRcl', class_id=4)
                      ])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [12]:
save_model_path = '/kaggle/working/checkpoint_7'

cp = tf.keras.callbacks.ModelCheckpoint(
    filepath=save_model_path, 
    monitor='val_MeanIoU', 
    mode='max', 
    save_best_only=True
)
cb_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_MeanIoU',
    mode='max',
    min_delta=0.001,
    patience=4,
    verbose=1
)
cb_reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_MeanIoU',
    mode='min',
    factor=0.1,
    patience=4,
    verbose=1,
    min_lr=0.00001
)
    
history = model.fit(train.repeat(), 
                    steps_per_epoch=int(np.ceil(train_percent*len(image_paths) / float(BATCH_SIZE))),
                    epochs=epochs,
                    validation_data=val.repeat(),
                    validation_steps=int(np.ceil(val_percent*len(image_paths) / float(BATCH_SIZE))),
                    callbacks=[cp, cb_earlystop, cb_reducelr],
                    verbose=1)

Epoch 1/15


2022-07-24 12:21:11.521184: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-24 12:21:11.522553: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000150000 Hz
2022-07-24 12:21:18.744370: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-07-24 12:21:24.922184: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-07-24 12:21:25.775522: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


1765/1765 [==============================] - 1702s 955ms/step - loss: 0.5579 - MeanIoU: 0.3741 - bgrPcsn: 0.8632 - bldPcsn: 0.4754 - wldPcsn: 0.7593 - wtrPcsn: 0.5451 - roadPcsn: 0.3689 - bgrRcl: 0.7517 - bldRcl: 0.1222 - wldRcl: 0.8079 - wtrRcl: 0.2918 - roadRcl: 0.1370 - val_loss: 0.4863 - val_MeanIoU: 0.5087 - val_bgrPcsn: 0.9033 - val_bldPcsn: 0.6283 - val_wldPcsn: 0.8611 - val_wtrPcsn: 0.5857 - val_roadPcsn: 0.4984 - val_bgrRcl: 0.8545 - val_bldRcl: 0.3566 - val_wldRcl: 0.8947 - val_wtrRcl: 0.6922 - val_roadRcl: 0.2404


2022-07-24 12:49:37.178994: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/15
1765/1765 [==============================] - 1592s 902ms/step - loss: 0.4854 - MeanIoU: 0.5433 - bgrPcsn: 0.9064 - bldPcsn: 0.6397 - wldPcsn: 0.8413 - wtrPcsn: 0.7345 - roadPcsn: 0.4996 - bgrRcl: 0.8560 - bldRcl: 0.4323 - wldRcl: 0.8949 - wtrRcl: 0.6723 - roadRcl: 0.3290 - val_loss: 0.4952 - val_MeanIoU: 0.5040 - val_bgrPcsn: 0.9526 - val_bldPcsn: 0.2157 - val_wldPcsn: 0.8628 - val_wtrPcsn: 0.7838 - val_roadPcsn: 0.1990 - val_bgrRcl: 0.7453 - val_bldRcl: 0.8728 - val_wldRcl: 0.9190 - val_wtrRcl: 0.8131 - val_roadRcl: 0.4306
Epoch 3/15
1765/1765 [==============================] - 1585s 898ms/step - loss: 0.4658 - MeanIoU: 0.6028 - bgrPcsn: 0.9247 - bldPcsn: 0.6782 - wldPcsn: 0.8625 - wtrPcsn: 0.7925 - roadPcsn: 0.5372 - bgrRcl: 0.8754 - bldRcl: 0.5792 - wldRcl: 0.9156 - wtrRcl: 0.7607 - roadRcl: 0.4059 - val_loss: 0.4787 - val_MeanIoU: 0.5617 - val_bgrPcsn: 0.9433 - val_bldPcsn: 0.8354 - val_wldPcsn: 0.8254 - val_wtrPcsn: 0.5465 - val_roadPcsn: 0.7021 - val_bgrRcl: 0.8009 - v

In [13]:
model.save('/kaggle/working/model_7')
!zip -r model_7.zip /kaggle/working/model_7

  adding: kaggle/working/model_7/ (stored 0%)
  adding: kaggle/working/model_7/variables/ (stored 0%)
  adding: kaggle/working/model_7/variables/variables.index (deflated 80%)
  adding: kaggle/working/model_7/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: kaggle/working/model_7/assets/ (stored 0%)
  adding: kaggle/working/model_7/saved_model.pb (deflated 92%)


In [14]:
# model = tf.keras.models.load_model('/kaggle/input/model6/model_6/', compile=False)

In [15]:
loss, acc, *is_anything_else_being_returned = model.evaluate(test, verbose=1, batch_size=BATCH_SIZE)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

589/589 [==============================] - 218s 276ms/step - loss: 0.4174 - MeanIoU: 0.7445 - bgrPcsn: 0.9597 - bldPcsn: 0.7762 - wldPcsn: 0.9054 - wtrPcsn: 0.9392 - roadPcsn: 0.6939 - bgrRcl: 0.9286 - bldRcl: 0.7727 - wldRcl: 0.9482 - wtrRcl: 0.9181 - roadRcl: 0.5962
Restored model, accuracy: 74.45%


In [16]:
# import matplotlib.pyplot as plt
# def display(display_list):
#     plt.figure(figsize=(15, 15))

#     title = ['Input Image', 'True Mask', 'Predicted Mask']

#     for i in range(len(display_list)):
#         plt.subplot(1, len(display_list), i+1)
#         plt.title(title[i])
#         plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
#         plt.axis('off')
#     plt.show()

# def create_mask(mask):
#     condensed_mask = tf.math.argmax(mask, axis=-1)
#     condensed_mask = condensed_mask[..., tf.newaxis]
#     return condensed_mask[0]

# def show_predictions(model, dataset=None, num=1):
#     for image, mask in dataset.take(num):
#         pred_mask = model.predict(image)
#         display([image[0], create_mask(mask), create_mask(pred_mask)])

# show_predictions(model, test, 35)

In [17]:
# import cv2
# import tensorflow as tf
# import matplotlib.pyplot as plt

# img_name = 'N-33-60-D-c-4-2'
# IMAGE_PATH = '../input/landcoverai/images/{}.tif'.format(img_name)
# LABEL_PATH = '../input/landcoverai/masks/{}.tif'.format(img_name)

# def from_one_hot_to_rgb_bkup(class_indexes, palette=None):
#     """ 
#     https://stackoverflow.com/a/60811084/6328456
#     Assign a different color to each class in the input tensor 
#     """
#     # 0 is background, 1 is building, 2 is woodland, 3 is water, 4 is road
#     if palette is None:
#         palette = tf.constant(
#             [[0, 0, 0], #background - black
#             [128, 87, 43], #building - brown
#             [12, 243, 12], #woodland - light green
#             [12, 122, 251], #water - sky blue
#             [79, 12, 75]] #road - dark purple
#         , dtype=tf.int32)

#     H, W, _ = class_indexes.shape
#     class_indexes = tf.cast(class_indexes, tf.int32)

#     color_image = tf.gather(palette, class_indexes)
#     color_image = tf.reshape(color_image, [H, W, 3])

#     color_image = tf.cast(color_image, dtype=tf.float32)
#     return color_image

# def pad_image_to_tile_multiple(image3, tile_size, padding="CONSTANT"):
#     '''
#     https://stackoverflow.com/a/46181172/6328456
#     Pad an image to a multiple of the tile size.
#     Input:
#         image3: A 3D tensor with shape (H,W,C)
#         tile_size: Tuple denoting size of the tiles.
#         padding: Padding type for tf.pad command.
#     Returns:
#         A padded tensor.
#     '''
#     original_image_size = image3.shape
#     image_size = tf.shape(image3)[0:2]
#     padding_ = tf.cast(tf.math.ceil(image_size / tile_size), tf.int32) * tile_size - image_size
#     return original_image_size, tf.pad(image3, [[0, padding_[0]], [0, padding_[1]], [0, 0]], padding)

# def remove_image_padding(image3, original_img_shape):
#     image3 = image3[0:original_img_shape[0], 0:original_img_shape[1], :]
#     return image3

# def split_image(image3, tile_size):
#     '''
#     https://stackoverflow.com/a/46181172/6328456
#     Split an image into tiles. Converts a 3D tensor with shape (H,W,C) to a 4D tensor with shape (B,H,W,C).
#     Input:
#         image3: A 3D tensor with shape (H,W,C)
#         tile_size: Tuple denoting size of the tiles.
#     Returns:
#         A 4D tensor with shape (B,H,W,C)
#     '''
#     image_shape = tf.shape(image3)
#     tile_rows = tf.reshape(image3, [image_shape[0], -1, tile_size[1], image_shape[2]])
#     serial_tiles = tf.transpose(tile_rows, [1, 0, 2, 3])
#     return tf.reshape(serial_tiles, [-1, tile_size[1], tile_size[0], image_shape[2]])

# def unsplit_image(tiles4, image_shape):
#     '''
#     https://stackoverflow.com/a/46181172/6328456
#     Unsplit a tiles into an image. Converts a 4D tensor with [B,H,W,C] to a 3D tensor with [H,W,C].
#     Input:
#         tiles4: A 4D tensor with shape (B,H,W,C)
#         image_shape: Tuple denoting size of the image.
#     Returns:
#         A 3D tensor with shape (H,W,C)
#     '''
#     tile_width = tf.shape(tiles4)[1]
#     serialized_tiles = tf.reshape(tiles4, [-1, image_shape[0], tile_width, image_shape[2]])
#     rowwise_tiles = tf.transpose(serialized_tiles, [1, 0, 2, 3])
#     return tf.reshape(rowwise_tiles, [image_shape[0], image_shape[1], image_shape[2]])

# # Read image and convert to tensor
# img = cv2.imread(IMAGE_PATH)
# label = cv2.imread(LABEL_PATH, cv2.IMREAD_UNCHANGED)
# img = tf.convert_to_tensor(img, tf.float32)
# img = img / 255.0
# label = tf.convert_to_tensor(label, tf.float32)
# label = label[..., tf.newaxis]

# # Pad the image and label
# original_label_shape, _ = pad_image_to_tile_multiple(label, [256, 256])
# padded_label_shape = _.shape
# original_img_shape, img = pad_image_to_tile_multiple(img, [256, 256])

# # Split the padded image into batches and pad the batches
# tiles = split_image(img, [256, 256])

# pred_masks = model.predict(tiles)
# pred_masks = tf.math.argmax(pred_masks, axis=-1)
# pred_masks = pred_masks[...,tf.newaxis]
# pred_mask = unsplit_image(pred_masks, padded_label_shape)

# # Remove the padding from the image and mask
# img = remove_image_padding(img, original_img_shape)
# pred_mask = remove_image_padding(pred_mask, original_label_shape)

# # Convert the masks to a uint8 image
# label = from_one_hot_to_rgb_bkup(label)
# pred_mask = from_one_hot_to_rgb_bkup(pred_mask)
# label = tf.keras.utils.array_to_img(label)
# pred_mask = tf.keras.utils.array_to_img(pred_mask)

# # Save as png
# # img = tf.io.encode_png(tf.cast(img, tf.uint8))
# # tf.io.write_file('{}_0.png'.format(img_name), img)
# label.save('{}_label.png'.format(img_name))
# pred_mask.save('{}_predMask.png'.format(img_name))